In [1]:
import pandas as pd
import numpy as np

# data = np.load('distorted_data/concatenated_data.npz')
data = np.load('datasets/training.npz')
print(len(data['original']), list(data.keys()))
data

# # load the data from every file in the distorted_data directory

# import os
# data_dir = 'distorted_data'

# data = []
# for filename in os.listdir(data_dir):
#     if filename.endswith('.npz'):
#         file_path = os.path.join(data_dir, filename)
#         loaded_data = np.load(file_path)
#         data.append(loaded_data)

# # concatenate the data from all files
# all_data = {}
# for key in data[0].keys():
#     all_data[key] = np.concatenate([d[key] for d in data], axis=0)

# # check the shape of the concatenated data
# for key, value in all_data.items():
#     print(f"{key}: {value.shape}")

# # save the concatenated data to a new npz file
# output_file = 'distorted_data/concatenated_data.npz'
# np.savez(output_file, **all_data)

34561 ['original', 'label', 'Uniform_Noise', 'Rotate_90deg']


In [2]:
import os

def import_data(directory, save_path=None, save=False):

    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.npz'):
            file_path = os.path.join(directory, filename)
            loaded_data = np.load(file_path)
            data.append(loaded_data)

    # concatenate the data from all files
    all_data = {}
    for key in data[0].keys():
        all_data[key] = np.concatenate([d[key] for d in data], axis=0)

    # check the shape of the concatenated data
    for key, value in all_data.items():
        print(f"{key}: {value.shape}")  

    if save:
        if save_path is None:
            save_path = f'datasets/{directory}_concatenated_data.npz'
        np.savez(save_path, **all_data)
        print(f"Data saved to {save_path}")

    return all_data

In [3]:
train_data = import_data('imported_distorted/train', save_path='datasets/train.npz', save=True)
val_data =  import_data('imported_distorted/val', save_path='datasets/validation.npz', save=True)
test_data = import_data('imported_distorted/test', save_path='datasets/test.npz', save=True)

# Shuffle the training data
seed = 42
np.random.seed(seed)

# Shuffle the data
train_indices = np.random.permutation(len(train_data['original']))
val_indices = np.random.permutation(len(val_data['original']))
test_indices = np.random.permutation(len(test_data['original']))


for key in train_data.keys():
    train_data[key] = train_data[key][train_indices]

for key in val_data.keys():
    val_data[key] = val_data[key][val_indices]

for key in test_data.keys():
    test_data[key] = test_data[key][test_indices]


original: (34561, 224, 224)
label: (34561, 1)
Uniform_Noise: (34561, 224, 224)
Rotate_90deg: (34561, 224, 224)
Data saved to datasets/train.npz
original: (6491, 224, 224)
label: (6491, 1)
Uniform_Noise: (6491, 224, 224)
Rotate_90deg: (6491, 224, 224)
Data saved to datasets/validation.npz
original: (17778, 224, 224)
label: (17778, 1)
Uniform_Noise: (17778, 224, 224)
Rotate_90deg: (17778, 224, 224)
Data saved to datasets/test.npz


In [4]:
test_data.keys()

dict_keys(['original', 'label', 'Uniform_Noise', 'Rotate_90deg'])

In [5]:
# keys = list(data.keys())

# images = [data[keys[0]], data[keys[2]], data[keys[3]]]
# labels = data[keys[1]]

In [6]:
# images[0][0]

In [7]:
def normalize_image(image, mean=0.5, std=0.5):
    """
    Normalize an image tensor to have a mean and standard deviation.
    """
    return (image - mean) / std

def normalize_images(images, mean=0.5, std=0.5):
    """
    Normalize a list of images.
    """
    return [normalize_image(image, mean, std) for image in images]

In [8]:
# normalized_images = []
# for image in images:
#     normalized_images.append(normalize_images(image))

In [9]:
def plot_sample_images(images, labels, n=3):
    """
    Plot a sample of images with their corresponding labels.
    """
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure(figsize=(15, 5))
    for i in range(n):
        plt.subplot(1, n, i + 1)
        plt.imshow(images[i][0], cmap='gray')
        plt.title(f'Label: {labels[i]}')
        plt.axis('off')
    plt.show()

# plot_sample_images(images, labels)
# plot_sample_images(normalized_images, labels)

In [10]:
# images[0][0]

In [11]:
# normalized_images[0][0]

In [12]:
# type(normalized_images[0][0])
# normalized_images[0][0].shape

In [13]:
# zero_labels = np.zeros_like(labels)
# one_labels = np.ones_like(labels)

# print(zero_labels.shape, one_labels.shape)

# domain_labels = np.concatenate((zero_labels, one_labels, one_labels), axis=0)
# print(domain_labels.shape)

# expanded_labels = np.concatenate((labels, labels, labels), axis=0)
# print(expanded_labels.shape)

In [14]:
# # concatenate the matrices in the images list
# concatenated_images = np.concatenate(normalized_images, axis=0)
# concatenated_images.shape

In [15]:
from branched_resnet import CustomImageDataset

def preprocess_data(data, distortions = [3], include_original=True, save_data=False, save_path=None):

    keys = list(data.keys())

    if include_original:
        images = [data[keys[0]]]
    else:
        images = []

    for distortion in distortions:
        images.append(data[keys[distortion]])

    labels = data[keys[1]]

    normalized_images = []
    for image in images:
        normalized_images.append(normalize_images(image))

    zero_labels = np.zeros_like(labels)
    one_labels = np.ones_like(labels)

    if include_original:
        domain_label_list = [zero_labels]
        expanded_label_list = [labels]
    else:
        domain_label_list = []
        expanded_label_list = []

    for _ in distortions:
        domain_label_list.append(one_labels)
        expanded_label_list.append(labels)

    domain_labels = np.concatenate(domain_label_list, axis=0)
    expanded_labels = np.concatenate(expanded_label_list, axis=0)

    print(f"Domain labels shape: {domain_labels.shape}")
    print(f"Expanded labels shape: {expanded_labels.shape}")

    concatenated_images = np.concatenate(normalized_images, axis=0)

    # Shuffle the concatenated images and labels
    seed = 42
    np.random.seed(seed)
    shuffled_indices = np.random.permutation(len(concatenated_images))
    concatenated_images = concatenated_images[shuffled_indices]
    expanded_labels = expanded_labels[shuffled_indices]
    domain_labels = domain_labels[shuffled_indices]

    if save_data:
        if save_path is None:
            raise ValueError("save_path must be specified if save_data is True")
        np.savez_compressed(save_path, images=concatenated_images, labels1=expanded_labels, labels2=domain_labels)

    dataset = CustomImageDataset(images=concatenated_images, labels1=expanded_labels, labels2=domain_labels)

    return dataset

In [16]:
import branched_resnet_v2 as br 
from branched_resnet_v2 import CustomImageDataset
# import branched_resnet as br 
# from branched_resnet import CustomImageDataset

from transformers import Trainer, TrainingArguments, PreTrainedModel, ResNetConfig

config = ResNetConfig()
model = br.ResNetForMultiLabel(config=config, num_d1_classes=11, num_d2_classes=2, lamb = 0)

for name, param in model.named_parameters():
    print(name, param.data.mean(), param.requires_grad)
    break

#train_ds = CustomImageDataset(images=concatenated_images, labels1=expanded_labels, labels2=domain_labels)

# vars(model)

resnet.embedder.embedder.convolution.weight tensor(-1.7982e-05) True


In [17]:
train_ds = preprocess_data(train_data, distortions=[3], save_data=True, save_path='datasets/D20_v2_processed_train_data.npz')
val_ds = preprocess_data(val_data, distortions=[], save_data=True, save_path='datasets/D20_v2_processed_val_data.npz')
#test_ds = preprocess_data(test_data, distortions=[2, 3])

Domain labels shape: (69122, 1)
Expanded labels shape: (69122, 1)
Domain labels shape: (6491, 1)
Expanded labels shape: (6491, 1)


In [18]:
type(val_ds)

branched_resnet_v2.CustomImageDataset

# D20

Train on undistorted and rotate 90

Test on the rest

In [19]:
# Uncomment to train the model
trainer = br.train_model(train_dataset=train_ds, eval_dataset= val_ds, model=model, output_dir= "./D20_Experiment", num_epochs=50, batch_size=32)
trainer.save_model('branched_resnet_model_D20_v2_0807')

C:\Users\samue\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: samuelsavine (samuelsavine-johns-hopkins-university). Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Accuracy Branch1,Precision Branch1,Recall Branch1,F1 Branch1,Accuracy Branch2,Precision Branch2,Recall Branch2,F1 Branch2,Lambda
1,1.081600,0.913724,0.809120,0.738810,0.746397,0.706368,0.768141,0.500000,0.384070,0.434434,0.000000
2,1.321300,0.666521,0.904791,0.898811,0.879645,0.876796,0.991219,0.500000,0.495609,0.497795,0.099668
3,2.273700,2.494968,0.920659,0.917688,0.906091,0.905602,0.000000,0.000000,0.000000,0.000000,0.197375
4,3.737300,1.758021,0.879217,0.907207,0.883580,0.886771,0.064089,0.500000,0.032044,0.060229,0.291313
5,5.644900,0.273679,0.928670,0.918629,0.908133,0.906721,1.000000,1.000000,1.000000,1.000000,0.379949
6,6.015900,7.905061,0.917732,0.914086,0.889887,0.886007,0.000000,0.000000,0.000000,0.000000,0.462117
7,7.144300,0.421600,0.868895,0.889770,0.861603,0.867109,1.000000,1.000000,1.000000,1.000000,0.537050
8,9.133400,8.716675,0.896934,0.899942,0.878330,0.878631,0.000000,0.000000,0.000000,0.000000,0.604368
9,10.700600,6.000147,0.486058,0.575509,0.552276,0.487792,1.000000,1.000000,1.000000,1.000000,0.664037
10,10.887300,16.727074,0.648590,0.826877,0.607465,0.586025,0.000000,0.000000,0.000000,0.000000,0.716298


In [20]:
# Uncomment to Load the model from checkpoint
# from safetensors.torch import load_file

# model_path = 'branched_resnet_v2_model_D20/model.safetensors'

# state_dict = load_file(model_path)
# model.load_state_dict(state_dict)

# trainer = br.train_model(train_dataset=train_ds, eval_dataset= val_ds, model=model, output_dir= "./D20_Experiment", num_epochs=50, batch_size=32, train=False)

In [24]:
# Clear up memory
train_data = None
val_data = None
train_ds = None
test_ds1 = preprocess_data(test_data, distortions=[2, 3], save_data=True, save_path='datasets/D20_v2_processed_test_data1.npz')
test_ds2 = preprocess_data(test_data, distortions=[2], save_data=True, save_path='datasets/D20_v2_processed_test_data2.npz')

Domain labels shape: (53334, 1)
Expanded labels shape: (53334, 1)
Domain labels shape: (35556, 1)
Expanded labels shape: (35556, 1)


In [25]:
baseline = preprocess_data(test_data, distortions=[], save_data=True, save_path='datasets/D20_v2_processed_baseline.npz')

Domain labels shape: (17778, 1)
Expanded labels shape: (17778, 1)


In [26]:
# Undistorted only
trainer.evaluate(baseline)

{'eval_loss': 0.5699331164360046,
 'eval_accuracy_branch1': 0.8309708628642142,
 'eval_precision_branch1': 0.8314732071658967,
 'eval_recall_branch1': 0.8114329511369401,
 'eval_f1_branch1': 0.7955323183361387,
 'eval_accuracy_branch2': 1.0,
 'eval_precision_branch2': 1.0,
 'eval_recall_branch2': 1.0,
 'eval_f1_branch2': 1.0,
 'eval_lambda': 0.9998891029505543,
 'eval_runtime': 27.2676,
 'eval_samples_per_second': 651.983,
 'eval_steps_per_second': 81.525,
 'epoch': 50.0}

In [ ]:
# Uniform, Rotate distortion and undistorted
trainer.evaluate(test_ds1)

{'eval_loss': 3.9435441493988037,
 'eval_accuracy_branch1': 0.6658791765102936,
 'eval_precision_branch1': 0.7568527037414916,
 'eval_recall_branch1': 0.6154068751186954,
 'eval_f1_branch1': 0.6337394254076444,
 'eval_accuracy_branch2': 0.3333333333333333,
 'eval_precision_branch2': 0.16666666666666666,
 'eval_recall_branch2': 0.5,
 'eval_f1_branch2': 0.25,
 'eval_lambda': 0.9998891029505543,
 'eval_runtime': 92.8103,
 'eval_samples_per_second': 574.656,
 'eval_steps_per_second': 71.835,
 'epoch': 50.0}

In [ ]:
# Undistorted and uniform noise
trainer.evaluate(test_ds2)

{'eval_loss': 5.866508960723877,
 'eval_accuracy_branch1': 0.5911238609517381,
 'eval_precision_branch1': 0.7365668863427057,
 'eval_recall_branch1': 0.5274228103316022,
 'eval_f1_branch1': 0.5574039248309851,
 'eval_accuracy_branch2': 0.5,
 'eval_precision_branch2': 0.25,
 'eval_recall_branch2': 0.5,
 'eval_f1_branch2': 0.3333333333333333,
 'eval_lambda': 0.9998891029505543,
 'eval_runtime': 61.2294,
 'eval_samples_per_second': 580.701,
 'eval_steps_per_second': 72.596,
 'epoch': 50.0}

In [29]:
test_ds3 = preprocess_data(test_data, distortions=[2, 3], include_original=False, save_data=True, save_path='datasets/D20_v2_processed_test_data3.npz')
test_ds4 = preprocess_data(test_data, distortions=[2], include_original=False, save_data=True, save_path='datasets/D20_v2_processed_test_data4.npz')

Domain labels shape: (35556, 1)
Expanded labels shape: (35556, 1)
Domain labels shape: (17778, 1)
Expanded labels shape: (17778, 1)


In [ ]:
# Uniform noise and rotate distortion
trainer.evaluate(test_ds3)

{'eval_loss': 3.0971689224243164,
 'eval_accuracy_branch1': 0.5833333333333334,
 'eval_precision_branch1': 0.7288149298575395,
 'eval_recall_branch1': 0.517393837109573,
 'eval_f1_branch1': 0.5415078906645008,
 'eval_accuracy_branch2': 0.0,
 'eval_precision_branch2': 0.0,
 'eval_recall_branch2': 0.0,
 'eval_f1_branch2': 0.0,
 'eval_lambda': 0.9998891029505543,
 'eval_runtime': 156.4692,
 'eval_samples_per_second': 227.24,
 'eval_steps_per_second': 28.408,
 'epoch': 50.0}

In [ ]:
# Uniform noise
trainer.evaluate(test_ds4)

{'eval_loss': 3.5957043170928955,
 'eval_accuracy_branch1': 0.35127685903926203,
 'eval_precision_branch1': 0.2233424711617616,
 'eval_recall_branch1': 0.2434126695262643,
 'eval_f1_branch1': 0.1749148339038518,
 'eval_accuracy_branch2': 0.0,
 'eval_precision_branch2': 0.0,
 'eval_recall_branch2': 0.0,
 'eval_f1_branch2': 0.0,
 'eval_lambda': 0.9998891029505543,
 'eval_runtime': 29.2237,
 'eval_samples_per_second': 608.342,
 'eval_steps_per_second': 76.068,
 'epoch': 50.0}

In [32]:
# Rotate distortion 
test_ds5 = preprocess_data(test_data, distortions=[3], include_original=False, save_data=True, save_path='datasets/D20_v2_processed_test_data5.npz')
trainer.evaluate(test_ds5)

Domain labels shape: (17778, 1)
Expanded labels shape: (17778, 1)


{'eval_loss': 2.5986340045928955,
 'eval_accuracy_branch1': 0.8153898076274047,
 'eval_precision_branch1': 0.8226051537439986,
 'eval_recall_branch1': 0.7913750046928816,
 'eval_f1_branch1': 0.7698344689556765,
 'eval_accuracy_branch2': 0.0,
 'eval_precision_branch2': 0.0,
 'eval_recall_branch2': 0.0,
 'eval_f1_branch2': 0.0,
 'eval_lambda': 0.9998891029505543,
 'eval_runtime': 29.7432,
 'eval_samples_per_second': 597.716,
 'eval_steps_per_second': 74.74,
 'epoch': 50.0}

In [33]:
def combine_npzs(data_dir):
    combined_data = {}
    order = ['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'last']
    # Process files in the order specified by the 'order' list
    for pos in order:
        for filename in os.listdir(data_dir):
            file_split = filename.split('_')
            if pos in file_split and filename.endswith('.npz'):
                print(f"Processing file: {filename}")
                file_path = os.path.join(data_dir, filename)
                data = np.load(file_path)
                for key in data.files:
                    if key in combined_data:
                        combined_data[key] = np.concatenate((combined_data[key], data[key]), axis=0)
                    else:
                        combined_data[key] = data[key]
    return combined_data

def combine_data(orig_data, new_data):
    combined_data = dict(orig_data)

    combined_data['Ring_Artifact_v1'] = new_data['Ring_Artifact_v1']
    combined_data['ring_labels'] = new_data['label']

    return combined_data

In [34]:
# CT Ring distortion

test_data = combine_npzs('ct-distortion/test')
orig_test_data = np.load('datasets/test.npz')
combined_test_data = combine_data(orig_test_data, test_data)

Processing file: test_subset_first_5000_RingArtifactv1_images.npz
Processing file: test_subset_second_5000_RingArtifactv1_images.npz
Processing file: test_subset_third_5000_RingArtifactv1_images.npz
Processing file: test_subset_last_2778_RingArtifactv1_images.npz


In [35]:
# Ring Artifact only
d6images = []
for image in combined_test_data['Ring_Artifact_v1']:
    d6images.append(normalize_image(image))

d6labels = combined_test_data['ring_labels']
d6domain_labels = np.ones_like(d6labels)

np.savez_compressed('datasets/D20_v2_processed_test_data6.npz', images=d6images, labels1=d6labels, labels2=d6domain_labels)

test_ds6 = br.CustomImageDataset(images=d6images, labels1=d6labels, labels2=d6domain_labels)

trainer.evaluate(test_ds6)

{'eval_loss': 2.645179033279419,
 'eval_accuracy_branch1': 0.7421532230847114,
 'eval_precision_branch1': 0.7261624466043838,
 'eval_recall_branch1': 0.7051710897088456,
 'eval_f1_branch1': 0.6771080368862996,
 'eval_accuracy_branch2': 0.0,
 'eval_precision_branch2': 0.0,
 'eval_recall_branch2': 0.0,
 'eval_f1_branch2': 0.0,
 'eval_lambda': 0.9998891029505543,
 'eval_runtime': 29.9647,
 'eval_samples_per_second': 593.299,
 'eval_steps_per_second': 74.187,
 'epoch': 50.0}